# 🚀 LSTM Optimization - Wind Offshore - Google Colab GPU Edition

**Systematisches LSTM-Tuning mit GPU-Beschleunigung für Wind Offshore**

**Setup:**
- Runtime → Change runtime type → GPU (T4 oder A100)
- ~10-50x schneller als CPU!

In [ ]:
# Check GPU
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("\n🚀 GPU should show above!")

In [ ]:
# Clone Repository
!git clone https://github.com/chradden/AdvancedTimeSeriesPrediction.git
%cd AdvancedTimeSeriesPrediction/energy-timeseries-project

In [ ]:
# Install Dependencies
!pip install -q pandas numpy matplotlib seaborn scikit-learn tensorflow keras pytorch-lightning darts

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import time

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)

# GPU Config
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU configured: {len(gpus)} device(s)")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU found - training will be slow!")

## ⚙️ Configuration - Wind Offshore

**Zeitreihe: Wind Offshore**
- Offshore-Windenergie-Erzeugung
- ⚠️ Beachte: 9-monatige Stillstandsperiode (Apr 2023 - Jan 2024)

In [ ]:
# Configuration
SERIES_NAME = 'wind_offshore'

print(f"📊 Zeitreihe: {SERIES_NAME.upper()}")
print(f"✅ Konfiguration abgeschlossen!")

In [ ]:
# Load processed data
train_df = pd.read_csv(f'data/processed/{SERIES_NAME}_train.csv')
val_df = pd.read_csv(f'data/processed/{SERIES_NAME}_val.csv')
test_df = pd.read_csv(f'data/processed/{SERIES_NAME}_test.csv')

print(f"📂 Loading data for: {SERIES_NAME.upper()}")
print(f"   Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")
print(f"   Columns: {train_df.columns.tolist()[:10]}...")

In [ ]:
# Determine value column
value_col = 'wind_offshore'
feature_cols = [c for c in train_df.columns if c not in ['timestamp', value_col]]

print(f"Value column: {value_col}")
print(f"Features: {len(feature_cols)}")

## 🔧 Prepare Data

In [ ]:
def create_sequences(data, target, seq_length):
    """Create sequences for LSTM"""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(target[i + seq_length])
    return np.array(X), np.array(y)

# Scale data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(train_df[feature_cols])
y_train = scaler_y.fit_transform(train_df[[value_col]])

X_val = scaler_X.transform(val_df[feature_cols])
y_val = scaler_y.transform(val_df[[value_col]])

X_test = scaler_X.transform(test_df[feature_cols])
y_test_orig = test_df[value_col].values

print(f"✅ Data scaled: X_train shape = {X_train.shape}")

In [ ]:
# Create sequences
seq_length = 24

X_train_seq, y_train_seq = create_sequences(X_train, y_train.flatten(), seq_length)
X_val_seq, y_val_seq = create_sequences(X_val, y_val.flatten(), seq_length)
X_test_seq, _ = create_sequences(X_test, np.zeros(len(X_test)), seq_length)
y_test_seq = y_test_orig[seq_length:]

print(f"✅ Sequences created:")
print(f"   X_train_seq: {X_train_seq.shape}")
print(f"   X_val_seq: {X_val_seq.shape}")
print(f"   X_test_seq: {X_test_seq.shape}")

## 🧪 Experiment 1: Baseline LSTM

In [ ]:
# Build Baseline LSTM
model = keras.Sequential([
    layers.LSTM(64, activation='relu', return_sequences=False, input_shape=(seq_length, len(feature_cols))),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse')

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
]

print("✅ Baseline LSTM model built")
model.summary()

In [ ]:
# Train Baseline LSTM
print("🚀 Training Baseline LSTM...")
start = time.time()

history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

train_time = time.time() - start
print(f"\n✅ Training completed in {train_time:.1f}s ({train_time/60:.1f} min)")

In [ ]:
# Evaluate Baseline LSTM
y_pred_scaled = model.predict(X_test_seq, verbose=0)
y_pred = scaler_y.inverse_transform(y_pred_scaled).flatten()

rmse = np.sqrt(mean_squared_error(y_test_seq, y_pred))
mae = mean_absolute_error(y_test_seq, y_pred)
r2 = r2_score(y_test_seq, y_pred)

print(f"\n📊 BASELINE LSTM RESULTS:")
print(f"   R² = {r2:.4f}")
print(f"   RMSE = {rmse:.2f} MW")
print(f"   MAE = {mae:.2f} MW")
print(f"   Time = {train_time:.1f}s")

## 🧪 Experiment 2: Bi-LSTM

In [ ]:
# Build Bi-LSTM
model_bilstm = keras.Sequential([
    layers.Bidirectional(layers.LSTM(64, activation='relu', return_sequences=True), 
                        input_shape=(seq_length, len(feature_cols))),
    layers.Dropout(0.2),
    layers.Bidirectional(layers.LSTM(32, activation='relu')),
    layers.Dropout(0.2),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])

model_bilstm.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse')

print("✅ Bi-LSTM model built")
model_bilstm.summary()

In [ ]:
# Train Bi-LSTM
print("🚀 Training Bi-LSTM...")
start = time.time()

history_bilstm = model_bilstm.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

train_time_bilstm = time.time() - start
print(f"\n✅ Bi-LSTM training completed in {train_time_bilstm:.1f}s")

In [ ]:
# Evaluate Bi-LSTM
y_pred_bilstm_scaled = model_bilstm.predict(X_test_seq, verbose=0)
y_pred_bilstm = scaler_y.inverse_transform(y_pred_bilstm_scaled).flatten()

rmse_bilstm = np.sqrt(mean_squared_error(y_test_seq, y_pred_bilstm))
mae_bilstm = mean_absolute_error(y_test_seq, y_pred_bilstm)
r2_bilstm = r2_score(y_test_seq, y_pred_bilstm)

print(f"\n📊 BI-LSTM RESULTS:")
print(f"   R² = {r2_bilstm:.4f}")
print(f"   RMSE = {rmse_bilstm:.2f} MW")
print(f"   MAE = {mae_bilstm:.2f} MW")
print(f"   Time = {train_time_bilstm:.1f}s")

## 🧪 Experiment 3: Autoencoder-Forecast

In [ ]:
# Build Autoencoder
encoding_dim = 32

# Encoder
input_ae = layers.Input(shape=(seq_length, len(feature_cols)))
encoded = layers.LSTM(64, activation='relu', return_sequences=True)(input_ae)
encoded = layers.LSTM(encoding_dim, activation='relu')(encoded)

# Decoder
decoded = layers.RepeatVector(seq_length)(encoded)
decoded = layers.LSTM(64, activation='relu', return_sequences=True)(decoded)
decoded = layers.TimeDistributed(layers.Dense(len(feature_cols)))(decoded)

# Autoencoder
autoencoder = keras.Model(input_ae, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Encoder Model
encoder = keras.Model(input_ae, encoded)

print("✅ Autoencoder built")

In [ ]:
# Train Autoencoder
print("🚀 Training Autoencoder...")
start = time.time()

history_ae = autoencoder.fit(
    X_train_seq, X_train_seq,
    validation_data=(X_val_seq, X_val_seq),
    epochs=50,
    batch_size=64,
    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
    verbose=1
)

train_time_ae = time.time() - start
print(f"\n✅ Autoencoder training completed in {train_time_ae:.1f}s")

In [ ]:
# Train Forecast Head
encoded_train = encoder.predict(X_train_seq, verbose=0)
encoded_val = encoder.predict(X_val_seq, verbose=0)
encoded_test = encoder.predict(X_test_seq, verbose=0)

forecast_head = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=(encoding_dim,)),
    layers.Dense(1)
])

forecast_head.compile(optimizer='adam', loss='mse')

forecast_head.fit(
    encoded_train, y_train_seq,
    validation_data=(encoded_val, y_val_seq),
    epochs=50,
    batch_size=64,
    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
    verbose=0
)

print("✅ Forecast head trained")

In [ ]:
# Evaluate Autoencoder-Forecast
y_pred_ae_scaled = forecast_head.predict(encoded_test, verbose=0)
y_pred_ae = scaler_y.inverse_transform(y_pred_ae_scaled).flatten()

rmse_ae = np.sqrt(mean_squared_error(y_test_seq, y_pred_ae))
mae_ae = mean_absolute_error(y_test_seq, y_pred_ae)
r2_ae = r2_score(y_test_seq, y_pred_ae)

print(f"\n📊 AUTOENCODER-FORECAST RESULTS:")
print(f"   R² = {r2_ae:.4f}")
print(f"   RMSE = {rmse_ae:.2f} MW")
print(f"   MAE = {mae_ae:.2f} MW")
print(f"   Time = {train_time_ae:.1f}s")

## 🧪 Experiment 4: VAE-Forecast

In [ ]:
# Build VAE
latent_dim = 32

# Encoder
input_vae = layers.Input(shape=(seq_length, len(feature_cols)))
x = layers.LSTM(64, activation='relu', return_sequences=True)(input_vae)
x = layers.LSTM(64, activation='relu')(x)

z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

# Sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_input = layers.Input(shape=(latent_dim,))
x_decoded = layers.RepeatVector(seq_length)(decoder_input)
x_decoded = layers.LSTM(64, activation='relu', return_sequences=True)(x_decoded)
x_decoded = layers.TimeDistributed(layers.Dense(len(feature_cols)))(x_decoded)

decoder = keras.Model(decoder_input, x_decoded)
outputs = decoder(z)

vae = keras.Model(input_vae, outputs)

# VAE Loss
reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(input_vae, outputs), axis=-1))
kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
vae_loss = reconstruction_loss + kl_loss

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Encoder Model
encoder_vae = keras.Model(input_vae, z_mean)

print("✅ VAE built")

In [ ]:
# Train VAE
print("🚀 Training VAE...")
start = time.time()

history_vae = vae.fit(
    X_train_seq, X_train_seq,
    validation_data=(X_val_seq, X_val_seq),
    epochs=50,
    batch_size=64,
    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
    verbose=1
)

train_time_vae = time.time() - start
print(f"\n✅ VAE training completed in {train_time_vae:.1f}s")

In [ ]:
# Train Forecast Head for VAE
encoded_vae_train = encoder_vae.predict(X_train_seq, verbose=0)
encoded_vae_val = encoder_vae.predict(X_val_seq, verbose=0)
encoded_vae_test = encoder_vae.predict(X_test_seq, verbose=0)

forecast_head_vae = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=(latent_dim,)),
    layers.Dense(1)
])

forecast_head_vae.compile(optimizer='adam', loss='mse')

forecast_head_vae.fit(
    encoded_vae_train, y_train_seq,
    validation_data=(encoded_vae_val, y_val_seq),
    epochs=50,
    batch_size=64,
    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
    verbose=0
)

print("✅ VAE forecast head trained")

In [ ]:
# Evaluate VAE-Forecast
y_pred_vae_scaled = forecast_head_vae.predict(encoded_vae_test, verbose=0)
y_pred_vae = scaler_y.inverse_transform(y_pred_vae_scaled).flatten()

rmse_vae = np.sqrt(mean_squared_error(y_test_seq, y_pred_vae))
mae_vae = mean_absolute_error(y_test_seq, y_pred_vae)
r2_vae = r2_score(y_test_seq, y_pred_vae)

print(f"\n☕ VAE-FORECAST RESULTS:")
print(f"   R² = {r2_vae:.4f}")
print(f"   RMSE = {rmse_vae:.2f} MW")
print(f"   MAE = {mae_vae:.2f} MW")
print(f"   Time = {train_time_vae:.1f}s")

## 🧪 Experiment 5: N-BEATS (Advanced)

In [ ]:
# Install Darts if needed
!pip install -q darts

In [ ]:
# Import Darts
from darts import TimeSeries
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler as DartsScaler

# Create Darts TimeSeries
ts_train = TimeSeries.from_values(train_df[value_col].values)
ts_val = TimeSeries.from_values(val_df[value_col].values)
ts_test = TimeSeries.from_values(test_df[value_col].values)

# Scale
scaler_darts = DartsScaler()
ts_train_scaled = scaler_darts.fit_transform(ts_train)
ts_val_scaled = scaler_darts.transform(ts_val)

print("✅ Darts TimeSeries created")

In [ ]:
# Build N-BEATS
model_nbeats = NBEATSModel(
    input_chunk_length=24,
    output_chunk_length=1,
    n_epochs=100,
    batch_size=64,
    pl_trainer_kwargs={
        "accelerator": "gpu",
        "devices": 1,
        "enable_progress_bar": True
    },
    force_reset=True,
    save_checkpoints=True
)

print("🔧 N-BEATS model configured!")

In [ ]:
# Train N-BEATS
print("🚀 Training N-BEATS (this will be FAST on GPU!)...")
start = time.time()

model_nbeats.fit(
    series=ts_train_scaled,
    val_series=ts_val_scaled,
    verbose=True
)

train_time_nbeats = time.time() - start
print(f"\n✅ N-BEATS training completed in {train_time_nbeats:.1f}s ({train_time_nbeats/60:.1f} min)")

In [ ]:
# Predict with N-BEATS
n_pred = len(ts_test)
pred_nbeats_scaled = model_nbeats.predict(n=n_pred, series=ts_train_scaled)
pred_nbeats = scaler_darts.inverse_transform(pred_nbeats_scaled)

# Extract values
y_pred_nbeats = pred_nbeats.values().flatten()
y_test_nbeats = ts_test.values().flatten()

# Ensure same length
min_len = min(len(y_pred_nbeats), len(y_test_nbeats))
y_pred_nbeats = y_pred_nbeats[:min_len]
y_test_nbeats = y_test_nbeats[:min_len]

# Evaluate
rmse_nbeats = np.sqrt(mean_squared_error(y_test_nbeats, y_pred_nbeats))
mae_nbeats = mean_absolute_error(y_test_nbeats, y_pred_nbeats)
r2_nbeats = r2_score(y_test_nbeats, y_pred_nbeats)

print(f"\n📊 N-BEATS RESULTS:")
print(f"   R² = {r2_nbeats:.4f}")
print(f"   RMSE = {rmse_nbeats:.2f}")
print(f"   MAE = {mae_nbeats:.2f}")
print(f"   Time = {train_time_nbeats:.1f}s")

## 🧪 Experiment 6: N-HiTS (Advanced)

In [ ]:
# Import N-HiTS
from darts.models import NHiTSModel

# Build N-HiTS
model_nhits = NHiTSModel(
    input_chunk_length=24,
    output_chunk_length=1,
    n_epochs=100,
    batch_size=64,
    pl_trainer_kwargs={
        "accelerator": "gpu",
        "devices": 1,
        "enable_progress_bar": True
    },
    force_reset=True,
    save_checkpoints=True
)

print("🔧 N-HiTS model configured!")

In [ ]:
# Train N-HiTS
print("🚀 Training N-HiTS...")
start = time.time()

model_nhits.fit(
    series=ts_train_scaled,
    val_series=ts_val_scaled,
    verbose=True
)

train_time_nhits = time.time() - start
print(f"\n✅ N-HiTS training completed in {train_time_nhits:.1f}s ({train_time_nhits/60:.1f} min)")

In [ ]:
# Predict with N-HiTS
pred_nhits_scaled = model_nhits.predict(n=n_pred, series=ts_train_scaled)
pred_nhits = scaler_darts.inverse_transform(pred_nhits_scaled)

# Extract values
y_pred_nhits = pred_nhits.values().flatten()
y_test_nhits = ts_test.values().flatten()

# Ensure same length
min_len = min(len(y_pred_nhits), len(y_test_nhits))
y_pred_nhits = y_pred_nhits[:min_len]
y_test_nhits = y_test_nhits[:min_len]

# Evaluate
rmse_nhits = np.sqrt(mean_squared_error(y_test_nhits, y_pred_nhits))
mae_nhits = mean_absolute_error(y_test_nhits, y_pred_nhits)
r2_nhits = r2_score(y_test_nhits, y_pred_nhits)

print(f"\n📊 N-HiTS RESULTS:")
print(f"   R² = {r2_nhits:.4f}")
print(f"   RMSE = {rmse_nhits:.2f}")
print(f"   MAE = {mae_nhits:.2f}")
print(f"   Time = {train_time_nhits:.1f}s")

## 📊 Summary - All Models

Zusammenfassung aller Modelle für Wind Offshore

In [ ]:
# Summary DataFrame
results = pd.DataFrame([
    {'Model': 'Baseline LSTM', 'R²': r2, 'RMSE': rmse, 'MAE': mae, 'Time (s)': train_time},
    {'Model': 'Bi-LSTM', 'R²': r2_bilstm, 'RMSE': rmse_bilstm, 'MAE': mae_bilstm, 'Time (s)': train_time_bilstm},
    {'Model': 'Autoencoder', 'R²': r2_ae, 'RMSE': rmse_ae, 'MAE': mae_ae, 'Time (s)': train_time_ae},
    {'Model': 'VAE', 'R²': r2_vae, 'RMSE': rmse_vae, 'MAE': mae_vae, 'Time (s)': train_time_vae},
    {'Model': 'N-BEATS', 'R²': r2_nbeats, 'RMSE': rmse_nbeats, 'MAE': mae_nbeats, 'Time (s)': train_time_nbeats},
    {'Model': 'N-HiTS', 'R²': r2_nhits, 'RMSE': rmse_nhits, 'MAE': mae_nhits, 'Time (s)': train_time_nhits}
])

results = results.sort_values('R²', ascending=False)

print("\n" + "="*80)
print(f"📊 WIND OFFSHORE - ALL MODELS SUMMARY")
print("="*80)
print(results.to_string(index=False))
print("\n✅ Bestes Modell:", results.iloc[0]['Model'])
print(f"   R² = {results.iloc[0]['R²']:.4f}")

# Save results
results.to_csv('results/metrics/deep_learning_comprehensive_wind_offshore.csv', index=False)
print("\n💾 Ergebnisse gespeichert: results/metrics/deep_learning_comprehensive_wind_offshore.csv")